In [28]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=2
import numpy as np
import pandas as pd
import os
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import umap
from firelight.visualizers.colorization import get_distinct_colors
from matplotlib.colors import ListedColormap
import pickle
from umap.my_utils import acc_kNN, corr_pdist_subsample, \
    reproducing_loss_keops, expected_loss_keops, filter_graph, KL_divergence, \
    low_dim_sim_keops_dist, compute_low_dim_psim_keops_embd
import torch
import scipy.special

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: CUDA_VISIBLE_DEVICES=2


In [3]:
data_path_c_elegans = "../data/packer_c-elegans"
fig_path = "../figures"
seed = 0
repeats = 7
special_cell_type = "Seam_cell"

In [4]:
# load the data
pca100 = pd.read_csv(os.path.join(data_path_c_elegans,
                              "c-elegans_qc_final.txt"),
                     sep='\t',
                     header=None)
pca100.shape

(86024, 100)

In [5]:
# read meta data, obtain colors and ordering of cells that puts special_cell_type to the front and unlabelled cells to
# the back
meta = pd.read_csv(os.path.join(data_path_c_elegans,
                              "c-elegans_qc_final_metadata.txt"),
                   sep=',',
                   header=0)

cell_types = meta["cell.type"].to_numpy().astype(str)

labels = np.zeros(len(cell_types)).astype(int)
name_to_label = {}
for i, phase in enumerate(np.unique(cell_types)):
    name_to_label[phase] = i
    labels[cell_types==phase] = i

special_cell_label = name_to_label[special_cell_type]
unnamed_label = name_to_label["nan"]

np.random.seed(seed)
colors = get_distinct_colors(len(name_to_label))
np.random.shuffle(colors)
colors[special_cell_label] = [0,0,0]
colors[unnamed_label] = [0.5, 0.5, 0.5]
cmap = ListedColormap(colors)

special_order1 = np.argsort(labels == special_cell_label, kind="stable") # put idx of special label to the back
special_order2 = np.argsort(labels[special_order1] != unnamed_label, kind="stable") # put idx of unnamed label to the front
special_order = special_order1[special_order2]
special_order_no_nan = special_order[(labels==unnamed_label).sum():]

## PCA

In [6]:
# load / compute and save 2D PCA
try:
    pca2 = np.load(os.path.join(data_path_c_elegans, "PCA2D.np"))
except FileNotFoundError:
    pca_projector = PCA(n_components = 2)
    pca2 = pca_projector.fit_transform(np.array(pca100))
    np.save(os.path.join(data_path_c_elegans, "PCA2D.npy"), pca2)

## UMAP
We use the hyperparameter settings of Narayan et al.'s "Assessing single-cell transcriptomic variability through density-preserving data
visualization" paper (https://doi.org/10.1038/s41587-020-00801-7) but with high-dimensional cosine metric


### Log losses after full epoch

In [7]:
# can take long, approx repeats * 30 min
umappers_c_elegans_after = []
for repeat in range(repeats):
    try:
        with open(os.path.join(data_path_c_elegans, f"umapperns_after_seed_{repeat}.pkl"), "rb") as file:
            umapper = pickle.load((file))
    except FileNotFoundError:
        umapper= umap.UMAP(metric="cosine",
                             n_neighbors=30,
                             n_epochs=750,
                             log_losses="after",
                             log_samples=False,
                             random_state=repeat,
                             verbose=True)
        _ = umapper.fit_transform(pca100)
        with open(os.path.join(data_path_c_elegans, f"umapperns_after_seed_{repeat}.pkl"), "wb") as file:
            pickle.dump(umapper, file, pickle.HIGHEST_PROTOCOL)
    umappers_c_elegans_after.append(umapper)
    print(f"done with run {repeat}")


Tue Oct 19 10:11:59 2021 Building and compiling search function
done with run 0
Tue Oct 19 10:12:04 2021 Building and compiling search function
done with run 1
Tue Oct 19 10:12:08 2021 Building and compiling search function
done with run 2
Tue Oct 19 10:12:10 2021 Building and compiling search function
done with run 3
Tue Oct 19 10:12:12 2021 Building and compiling search function
done with run 4
Tue Oct 19 10:12:14 2021 Building and compiling search function
done with run 5
Tue Oct 19 10:12:16 2021 Building and compiling search function
done with run 6


In [8]:
# inverted high dim similarities


In [9]:
# get inverted similarities
inv_graphs = []
assert "umappers_c_elegans_after" in locals()
for umapper in umappers_c_elegans_after:
    inv_graph = umapper.graph_.copy()
    inv_graph.data[inv_graph.data < inv_graph.data.max() / float(750)] = 0
    inv_graph.eliminate_zeros()
    inv_graph.data = inv_graph.data.min() / inv_graph.data
    inv_graphs.append(inv_graph)

In [10]:
umappers_c_elegans_inv_after = []
for i, repeat in enumerate(range(repeats)):
    try:
        with open(os.path.join(data_path_c_elegans, f"umapperns_inv_seed_{repeat}.pkl"), "rb") as file:
            umapper = pickle.load((file))
    except FileNotFoundError:
        umapper= umap.UMAP(metric="cosine",
                             n_neighbors=30,
                             n_epochs=750,
                             graph=inv_graphs[i],
                             log_samples=False,
                             log_loses="after",
                             random_state=repeat,
                             verbose=True)
        _ = umapper.fit_transform(pca100)
        with open(os.path.join(data_path_c_elegans, f"umapperns_inv_seed_{repeat}.pkl"), "wb") as file:
            pickle.dump(umapper, file, pickle.HIGHEST_PROTOCOL)
    umappers_c_elegans_inv_after.append(umapper)
    print(f"done with run {repeat}")

done with run 0
done with run 1
done with run 2
done with run 3
done with run 4
done with run 5
done with run 6


In [14]:
#### Quality measures of the embedding



# correlation measures
sample_size = 10000

pear_rs = []
spear_rs = []
pear_rs_inv = []
spear_rs_inv = []
pear_rs_pca = []
spear_rs_pca = []
for i in range(repeats):
    pear_r, spear_r = corr_pdist_subsample(pca100.to_numpy(),
                                           umappers_c_elegans_after[i].embedding_,
                                           sample_size,
                                           seed=i,
                                           metric="cosine")
    pear_rs.append(pear_r)
    spear_rs.append(spear_r)

    pear_r_inv, spear_r_inv = corr_pdist_subsample(pca100.to_numpy(),
                                                   umappers_c_elegans_inv_after[i].embedding_,
                                                   sample_size,
                                                   seed=i,
                                                   metric="cosine")
    pear_rs_inv.append(pear_r_inv)
    spear_rs_inv.append(spear_r_inv)

    pear_r_pca, spear_r_pca = corr_pdist_subsample(pca100.to_numpy(),
                                                   pca2,
                                                   sample_size,
                                                   seed=i,
                                                   metric="cosine")
    pear_rs_pca.append(pear_r_pca)
    spear_rs_pca.append(spear_r_pca)
    print(f"Done with run {i}")

pear_rs = np.stack(pear_rs)
spear_rs = np.stack(spear_rs)

pear_rs_inv = np.stack(pear_rs_inv)
spear_rs_inv = np.stack(spear_rs_inv)

pear_rs_pca = np.stack(pear_rs_pca)
spear_rs_pca = np.stack(spear_rs_pca)


print(f"Pearson UMAP mean:  {pear_rs.mean()}")
print(f"Pearson UMAP std:   {pear_rs.std()}")
print("\n")
print(f"Spearman UMAP mean: {spear_rs.mean()}")
print(f"Spearman UMAP std:  {spear_rs.std()}")
print("\n\n")

print(f"Pearson UMAP inv mean:  {pear_rs_inv.mean()}")
print(f"Pearson UMAP inv std:   {pear_rs_inv.std()}")
print("\n")
print(f"Spearman UMAP inv mean: {spear_rs_inv.mean()}")
print(f"Spearman UMAP inv std:  {spear_rs_inv.std()}")
print("\n\n")

print(f"Pearson PCA mean: {pear_rs_pca.mean()}")
print(f"Pearson PCA std: {pear_rs_pca.std()}")
print("\n")
print(f"Spearman PCA mean: {spear_rs_pca.mean()}")
print(f"Spearman PCA std:  {spear_rs_pca.std()}")


Done with run 0
Done with run 1
Done with run 2
Done with run 3
Done with run 4
Done with run 5
Done with run 6
Pearson UMAP mean:  0.5620050589878101
Pearson UMAP std:   0.007777905551735342


Spearman UMAP mean: 0.55127261231681
Spearman UMAP std:  0.010513767720885974



Pearson UMAP inv mean:  0.6279333786674318
Pearson UMAP inv std:   0.00507284434829625


Spearman UMAP inv mean: 0.6066066148760314
Spearman UMAP inv std:  0.005514853218002877



Pearson PCA mean: 0.5770961794617044
Pearson PCA std: 0.00287972201111643


Spearman PCA mean: 0.6113548671086951
Spearman PCA std:  0.0033829865064611804


In [15]:
# kNN based measure
acc10 = []
acc10_inv = []
acc10_pca = []
acc30 = []
acc30_inv = []
acc30_pca = []
for i in range(repeats):
    # k=10
    acc10.append(acc_kNN(pca100.to_numpy(),
                         umappers_c_elegans_after[i].embedding_,
                         k=10,
                         metric="cosine"))
    acc10_inv.append(acc_kNN(pca100.to_numpy(),
                             umappers_c_elegans_inv_after[i].embedding_,
                             k=10,
                             metric="cosine"))
    acc10_pca.append(acc_kNN(pca100.to_numpy(),
                             pca2, k=10,
                             metric="cosine"))
    # k=30
    acc30.append(acc_kNN(pca100.to_numpy(),
                         umappers_c_elegans_after[i].embedding_,
                         k=30,
                         metric="cosine"))
    acc30_inv.append(acc_kNN(pca100.to_numpy(),
                             umappers_c_elegans_inv_after[i].embedding_,
                             k=30,
                             metric="cosine"))
    acc30_pca.append(acc_kNN(pca100.to_numpy(),
                             pca2, k=30,
                             metric="cosine"))

acc10 = np.stack(acc10)
acc10_inv = np.stack(acc10_inv)
acc10_pca = np.stack(acc10_pca)

acc30 = np.stack(acc30)
acc30_inv = np.stack(acc30_inv)
acc30_pca = np.stack(acc30_pca)

In [16]:
print(f"10-NN accuracy UMAP mean:     {acc10.mean()}")
print(f"10-NN accuracy UMAP std:      {acc10.std()}\n")
print(f"10-NN accuracy UMAP inv mean: {acc10_inv.mean()}")
print(f"10-NN accuracy UMAP inv std:  {acc10_inv.std()}\n")
print(f"10-NN accuracy PCA mean:      {acc10_pca.mean()}")
print(f"10-NN accuracy PCA std:       {acc10_pca.std()}\n")

print(f"30-NN accuracy UMAP mean:     {acc30.mean()}")
print(f"30-NN accuracy UMAP std:      {acc30.std()}\n")
print(f"30-NN accuracy UMAP inv mean: {acc30_inv.mean()}")
print(f"30-NN accuracy UMAP inv std:  {acc30_inv.std()}\n")
print(f"30-NN accuracy PCA mean:      {acc30_pca.mean()}")
print(f"30-NN accuracy PCA std:       {acc30_pca.std()}\n")

10-NN accuracy UMAP mean:     0.1562079021136958
10-NN accuracy UMAP std:      0.0009206317229737199

10-NN accuracy UMAP inv mean: 0.07943597135683066
10-NN accuracy UMAP inv std:  0.0005319517736495356

10-NN accuracy PCA mean:      0.005883241885985306
10-NN accuracy PCA std:       8.673617379884035e-19

30-NN accuracy UMAP mean:     0.25606475269360046
30-NN accuracy UMAP std:      0.0013758122006941637

30-NN accuracy UMAP inv mean: 0.18454700349404152
30-NN accuracy UMAP inv std:  0.0011973435631427232

30-NN accuracy PCA mean:      0.013973619765026814
30-NN accuracy PCA std:       0.0



In [17]:
# various loss values:

min_dist = 0.1
spread = 1.0
a, b= umap.umap_.find_ab_params(spread=spread, min_dist=min_dist)

In [18]:
def get_losses(embd, graph, a,b, negative_sample_rate=5):
    loss_a_reprod, \
    loss_r_reprod = reproducing_loss_keops(high_sim=graph.tocoo(),
                                                 embedding=embd,
                                                 a=a,
                                                 b=b)
    loss_total_reprod = loss_a_reprod + loss_r_reprod

    loss_a_exp, \
    loss_r_exp = expected_loss_keops(high_sim=graph.tocoo(),
                                           embedding=embd,
                                           a=a,
                                           b=b,
                                           negative_sample_rate=negative_sample_rate,
                                           push_tail=True)
    loss_total_exp = loss_a_exp + loss_r_exp


    KL_div_norm_pos = KL_divergence(high_sim=graph.tocoo(),
                           a=a,
                           b=b,
                           embedding=embd
                           )

    KL_div = KL_divergence(high_sim=graph.tocoo(),
                           a=a,
                           b=b,
                           embedding=embd,
                           norm_over_pos=False
                           )

    return loss_total_reprod, loss_total_exp, KL_div_norm_pos, KL_div



In [22]:
loss_reprod, loss_exp, KL_div_pos, KL_div, \
loss_reprod_inv, loss_exp_inv, KL_div_pos_inv, KL_div_inv, \
loss_reprod_pca, loss_exp_pca, KL_div_pos_pca, KL_div_pca = [], [], [], [], [], [], [], [], [], [], [], []

for i in range(repeats):
    graph = filter_graph(umappers_c_elegans_after[i].graph_,
                         umappers_c_elegans_after[i].n_epochs)

    l_reprod,\
    l_exp,\
    kl_div_pos,\
    kl_div = get_losses(umappers_c_elegans_after[i].embedding_,
                        graph,
                        a,
                        b)
    loss_reprod.append(l_reprod)
    loss_exp.append(l_exp)
    KL_div_pos.append(kl_div_pos)
    KL_div.append(kl_div)


    l_reprod_inv, \
    l_exp_inv, \
    kl_div_pos_inv,\
    kl_div_inv = get_losses(umappers_c_elegans_inv_after[i].embedding_,
                            graph,
                            a,
                            b)
    loss_reprod_inv.append(l_reprod_inv)
    loss_exp_inv.append(l_exp_inv)
    KL_div_pos_inv.append(kl_div_pos_inv)
    KL_div_inv.append(kl_div_inv)

    l_reprod_pca,\
    l_exp_pca, \
    kl_div_pos_pca, \
    kl_div_pca = get_losses(pca2.astype(np.single),
                            graph,
                            a,
                            b)
    loss_reprod_pca.append(l_reprod_pca)
    loss_exp_pca.append(l_exp_pca)
    KL_div_pos_pca.append(kl_div_pos_pca)
    KL_div_pca.append(kl_div_pca)

loss_reprod = np.stack(loss_reprod)
loss_exp = np.stack(loss_exp)
KL_div_pos = np.stack(KL_div_pos)
KL_div = np.stack(KL_div)

loss_reprod_inv = np.stack(loss_reprod_inv)
loss_exp_inv = np.stack(loss_exp_inv)
KL_div_pos_inv = np.stack(KL_div_pos_inv)
KL_div_inv = np.stack(KL_div_inv)

loss_reprod_pca = np.stack(loss_reprod_pca)
loss_exp_pca = np.stack(loss_exp_pca)
KL_div_pos_pca = np.stack(KL_div_pos_pca)
KL_div_pca = np.stack(KL_div_pca)


In [47]:
print("UMAP")
print(f"Original UMAP loss mean:  {loss_reprod.mean(): .2E}")
print(f"Original UMAP loss std:   {loss_reprod.std(): .2E}\n")
print(f"True UMAP loss mean:      {loss_exp.mean(): .2E}")
print(f"True UMAP loss std:       {loss_exp.std(): .2E}\n")
print(f"KL divergence pos mean:   {KL_div_pos.mean()}")
print(f"KL divergence pos std:    {KL_div_pos.std()}\n")
print(f"KL divergence mean:       {KL_div.mean()}")
print(f"KL divergence std:        {KL_div.std()}\n")
print("\n")

print("UMAP inv ")
print(f"Original UMAP loss mean:  {loss_reprod_inv.mean(): .2E}")
print(f"Original UMAP loss std:   {loss_reprod_inv.std(): .2E}\n")
print(f"True UMAP loss mean:      {loss_exp_inv.mean(): .2E}")
print(f"True UMAP loss std:       {loss_exp_inv.std(): .2E}\n")
print(f"KL divergence pos mean:   {KL_div_pos_inv.mean()}")
print(f"KL divergence pos std:    {KL_div_pos_inv.std()}\n")
print(f"KL divergence mean:       {KL_div_inv.mean()}")
print(f"KL divergence std:        {KL_div_inv.std()}\n")
print("\n")

print("PCA")
print(f"Original UMAP loss mean:  {loss_reprod_pca.mean(): .2E}")
print(f"Original UMAP loss std:   {loss_reprod_pca.std(): .2E}\n")
print(f"True UMAP loss mean:      {loss_exp_pca.mean(): .2E}")
print(f"True UMAP loss std:       {loss_exp_pca.std(): .2E}\n")
print(f"KL divergence pos mean:   {KL_div_pos_pca.mean()}")
print(f"KL divergence pos std:    {KL_div_pos_pca.std()}\n")
print(f"KL divergence mean:       {KL_div_pca.mean()}")
print(f"KL divergence std:        {KL_div_pca.std()}\n")




UMAP
Original UMAP loss mean:   3.64E+08
Original UMAP loss std:    1.62E+06

True UMAP loss mean:       3.21E+05
True UMAP loss std:        1.50E+03

KL divergence pos mean:   0.5488904394897656
KL divergence pos std:    0.0015397640600636783

KL divergence mean:       4.920207187227171
KL divergence std:        0.004043140910223455



UMAP inv 
Original UMAP loss mean:   4.32E+08
Original UMAP loss std:    3.12E+06

True UMAP loss mean:       3.85E+05
True UMAP loss std:        1.98E+03

KL divergence pos mean:   0.5765721749717366
KL divergence pos std:    0.0016865110484212856

KL divergence mean:       5.129103837766089
KL divergence std:        0.006548632030396337



PCA
Original UMAP loss mean:   5.37E+08
Original UMAP loss std:    6.09E+01

True UMAP loss mean:       1.24E+06
True UMAP loss std:        3.35E+01

KL divergence pos mean:   0.692656118561421
KL divergence pos std:    3.864658684472962e-05

KL divergence mean:       6.461372502290538
KL divergence std:        4.69

In [25]:
def get_low_sim_pos_edges(high_sim, embd, a, b):
    heads = high_sim.row
    tails = high_sim.col
    sq_dist_pos_edges = ((embd[heads]-embd[tails])**2).sum(-1)
    low_sim_pos_edges = low_dim_sim_keops_dist(sq_dist_pos_edges, a, b, squared=True)
    return low_sim_pos_edges

In [43]:
# sanity check KL pos and normal KL differ only be normalization of embedding sims
for i in range(repeats):
    graph = filter_graph(umappers_c_elegans_after[i].graph_,
                         umappers_c_elegans_after[i].n_epochs)
    # normalization by embedding sims that have positive input sim
    Z_pos = get_low_sim_pos_edges(graph.tocoo(),
                                  umappers_c_elegans_after[i].embedding_,
                                  a,
                                  b).sum()
    # normalization by all pairs of embedding sims
    Z = compute_low_dim_psim_keops_embd(umappers_c_elegans_after[i].embedding_,
                                        a,
                                        b).sum(1).cpu().numpy().sum()
    assert np.abs((KL_div_pos[i] - np.log(Z_pos)
                   - (KL_div[i] - np.log(Z)))) < 0.0006

